## About

TODO

## Microbial diversity and composition across various surface types 
## surface site taxa difference analysis code 2

Next: `variation_across_surface_type_valcano_plot.Rmd`

In [1]:
import pandas as pd
import numpy as np
import os

In [ ]:
DATASET = os.path.join(
    "..","..","data"
)
CHENNAI_DATASET = os.path.join(DATASET,"Chennai_data")
RESULTS = os.path.join("../../results/")
KIOSK_RESULTS = os.path.join(
    "../../results/actinium_chennai_kiosk_species_abd_0"
)
SUPPLEMENTARY = os.path.join(
    "../../results/supplementary"
)

In [22]:
def get_mean(df, column, column_to_ignore):
    """
        get_mean
        input:
            df = Dataframe.
            column = name of the column.
            column_to_ignore = remove the column name from analysis

        output:
            return a groupby mean
    """
    temp = df[[i for i in df.columns if i not in column_to_ignore]]
    return temp.groupby(column).mean()

def entire_workflow(Table, LABEL_COLUMN = "is_kiosk"):
    """
        entire_workflow
        input:

        output:
            
        Performs log2FC, makes the required CSV for valcano plot (a csv table with taxa all, adj p-value for all, log2FC)
    """
    (site,table) = Table
    taxa = "species"
    feature_df = pd.read_csv(
        os.path.join(RESULTS,f"chennai_{site}_species_abd_0.001", "features", "filtered_data.tsv"),
        sep="\t",
    )
    feature_df.rename(columns={"feature": "Samples"}, inplace=True)

    ## Metadata of Chennai for Kiosk
    metadata_df = pd.read_csv(os.path.join(CHENNAI_DATASET, "metadata_processed.csv"))[['Samples',LABEL_COLUMN]]

    ## Merge the datasets
    merge_df = feature_df.merge(metadata_df, on="Samples", how="inner")

    log_mean_df = np.log2(
        get_mean(
            merge_df, column=LABEL_COLUMN, column_to_ignore=["Samples", "select object"]
        )
    )
    mean_kiosk_df = get_mean(
            merge_df, column=LABEL_COLUMN, column_to_ignore=["Samples", "select object"]
        ).loc[site]
    log2_FC = log_mean_df.loc[site] - log_mean_df.loc["Other"]
    log2_FC.sort_values(inplace=True)
    log2_FC.index.name = "Taxa"
    mean_kiosk_df.index.name = "Taxa"
    log2_FC = log2_FC.to_frame()
    mean_kiosk_df = mean_kiosk_df.to_frame()
    #print(log2_FC.head())
    #print(mean_kiosk_df.head())
    mean_kiosk_df.rename(columns={site: "Mean_rel_abt_on_kiosk"}, inplace=True)
    log2_FC = log2_FC.merge(mean_kiosk_df, on = 'Taxa',how='inner')
    log2_FC.to_csv(os.path.join(RESULTS,f"chennai_{site}_species_abd_0.001", "log2fc.csv"))

    ## Combine items to make required CSV for valcano plots
    log2_FC = pd.read_csv(os.path.join(RESULTS,f"chennai_{site}_species_abd_0.001", "log2fc.csv"))
    log2_FC.rename(columns={"0": "Log2FC"}, inplace=True)

    ## Read the Masline2 all_results.csv
    all_res_ = pd.read_csv(
        os.path.join(RESULTS,f"chennai_{site}_species_abd_0.001", "all_results.tsv"), sep="\t"
    )
    all_res_ = all_res_.merge(log2_FC, left_on="feature", right_on="Taxa", how="inner")
    all_res_.to_csv(
        os.path.join(RESULTS,f"chennai_{site}_species_abd_0.001", f"valcano_{taxa}.csv"),
        index=False,
    )

    all_res_["abs_Log2FC"] = np.absolute(all_res_["Log2FC"]) 
    all_res_ = all_res_.query("qval < 0.05 and abs_Log2FC > 1")
    all_res_["Over-/under-representation"] = all_res_["Log2FC"].apply(lambda x: "UP" if x>1 else "DOWN" if x < -1 else "NEUTRAL")

    ## Prittyfy the table
    all_res_.rename(columns={"feature":"Species", "pval":"p-value", "qval":"P_adj"}, inplace = True)
    all_res_ = all_res_[["Species","p-value","P_adj","Log2FC","Over-/under-representation"]]
    all_res_.to_csv(
        os.path.join(SUPPLEMENTARY, f"supplementary_{table}.csv"),
        index=False,
        )

In [23]:
for i in [("Kiosk","S5","is_kiosk"),("Rod","S6","is_Rod")]:#["",""]:
    print("Working on",i)
    entire_workflow(i[:2],i[-1])

wokring on ('Kiosk', 'S5', 'is_kiosk')
wokring on ('Rod', 'S6', 'is_Rod')


C:\Users\yuvar\AppData\Local\Temp\ipykernel_33984\3391681066.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_res_["Over-/under-representation"] = all_res_["Log2FC"].apply(lambda x: "UP" if x>1 else "DOWN" if x < -1 else "NEUTRAL")
C:\Users\yuvar\AppData\Local\Temp\ipykernel_33984\3391681066.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_res_.rename(columns={"feature":"Species", "pval":"p-value", "qval":"P_adj"}, inplace = True)
C:\Users\yuvar\AppData\Local\Temp\ipykernel_33984\3391681066.py:92: SettingWithCopyWarning: 
A value is trying to be s